![Banner logo](https://raw.githubusercontent.com/CitrineInformatics/community-tools/master/templates/fig/citrine_banner_2.png)

# Experimental design with Citrination

In this notebook, we demonstrate how to use Citrination to select experiments (or simulations, or literature searches) to maximize the impact on an optimization problem.

## Case study: thermoelectrics

Given a list of thermoelectric candidates, how many experiments do we need to find the best one?

Here, _best_ will be with respect to the thermoelecric figure of merit, ZT:

$$ ZT = \frac{\sigma S^2 T}{\lambda}, $$

where $\sigma$ is the electrical conductivity, $S$ is the Seebeck coefficient, $T$ is the temperature, and $\lambda$ is the thermal conductivity.  Higher is better.

## Python package imports

In [ ]:
# Standard packages
from os import environ
from random import shuffle, seed
from math import sqrt

# Third-party packages
from scipy.special import erf
import pandas as pd
from citrination_client import CitrinationClient
from citrination_client import PifSystemQuery, PifSystemReturningQuery
from citrination_client import FieldQuery, ValueQuery, NameQuery
from citrination_client import PropertyQuery,DataQuery, DatasetQuery, ChemicalFieldQuery, Filter

## Getting the data

We begin by querying for the thermoelectric dataset, which is on Citrination [here](https://citrination.com/datasets/150888).  We want the formula and zT

In [ ]:
dataset_id = 150888

system_query = PifSystemQuery(
    chemical_formula=ChemicalFieldQuery(
        extract_as="formula"),
    properties=PropertyQuery(
        name=FieldQuery(
            filter=[Filter(equal="ZT")]),
        value=FieldQuery(
            extract_as="ZT")))

thermoelectric_query = PifSystemReturningQuery(
                        size=1000,
                        random_seed=0,
                        query=DataQuery(
                            dataset=DatasetQuery(
                                id=[Filter(equal='150888')]),
                        system=system_query))

Let's run it and see what we get:

In [ ]:
client = CitrinationClient(api_key=environ['CITRINATION_API_KEY'], site='https://citrination.com')
search_result = client.search.pif_search(thermoelectric_query)
print("We found {} records".format(search_result.total_num_hits))
print([x.extracted for x in search_result.hits[0:2]])

This dataset has all the zT values already, so we want to drop most of them before trying to design an experiment:

In [ ]:
seed(1)
full_data = [x.extracted.copy() for x in search_result.hits]
shuffle(full_data)
known_subset = full_data[:20]
unknown_subset = full_data[20:]
for x in unknown_subset: 
    del x['ZT']

Our goal is to pick the best material to measure next from `unknown_subset`.

## Training a model on the known data

We train a model using the CSV -> dataset -> data view workflow described in the [ML on Citrination tutorial](tutorial_sequence/4_MLonCitrination.ipynb).

### Create a CSV

The CSV needs headers that conform to our [CSV template](http://help.citrination.com/knowledgebase/articles/1188136).

In [ ]:
def write_csv(name, rows):
    with open(name, "w") as f:
        f.write("FORMULA, PROPERTY: ZT \n")
        for row in rows:
            f.write("{formula:s}, {ZT:s}\n".format(**row))

write_csv('known_thermoelectrics.csv', known_subset)

The rest of the model building process is on the website:
 1. Go to the [Add Datasets](https://citrination.com/add_data) page and upload `known_thermoelectrics.csv` using the `Citrine: Template CSV` ingester from the drop down menu.
 1. Go to the [Data Views page](https://citrination.com/data_views) and click "Create new data view."
 1. Search for the property name "ZT" and select the dataset you created before.  Advance with the "NEXT >" button in the top right corner.
 1. Follow the guide to create a data view that has `formula` as an input and `ZT` as an output.

## Apply the model to unknown data

First, we'll use the trained model to make predictions via the API.  Change the `view_id` below to point to your view.

In [ ]:
view_id = "3904"
inputs = [{"formula": "TiO2"}]
prediction = client.models.predict(view_id, inputs, method="scalar")

print("The unknown material is {}.".format(inputs[0]['formula']))
print("We predict the ZT of {0} (@ 300K) to be {1:.4f} +/- {2:.4f}.".format(
    inputs[0]['formula'],
    prediction[0].get_value('Property ZT').value,
    prediction[0].get_value('Property ZT').loss))

### Maximum likelihood of improvement

This tutorial is about _experimental design_, so we need to pick a criterion for experimental selection.

There are many, but a straight forward and powerful one is "maximum likelihood of improvement," which is easy to compute if we assume the output distribution is normal.

In [ ]:
def probability_improvement(mean, sigma, baseline):
    return float(0.5 * (1.0 + erf((mean - baseline) / (sigma * sqrt(2.0)))))

What is the baseline?  The largest value in the known data:

In [ ]:
baseline_ZT = max(float(x['ZT']) for x in known_subset)
print("The highest ZT value in the known subset is {}.".format(baseline_ZT))

Now let's screen the unknown materials for likelihood of improvement:

In [ ]:
predictions = client.models.predict(view_id, unknown_subset)
for p in predictions:
    p_value = p.get_value('Property ZT')
    p.add_value('LI', probability_improvement(float(p_value.value), float(p_value.loss), baseline_ZT))

Pandas can help us look at the result.  The top values for "LI" are the ones we should try next.

In [ ]:
top_predictions = []
for p in predictions:
    li, form, zt = p.get_value('LI'), p.get_value('formula'), p.get_value('Property ZT')
    if li > 1e-03:
        top_predictions.append((form.value, [zt.value, zt.loss], li))
        
df = pd.DataFrame(top_predictions, columns=['formula', 'Property ZT', 'LI'])
df['Property ZT'] = df['Property ZT'].map(lambda x: "{:5.2f} +/- {:5.2f}".format(*x))
df['LI'] = df['LI'].map(lambda x: "{:5.3f}".format(x))

df.sort_values('LI', axis=0, ascending=False).head()

These materials have a likelihood of improvement below 50%, i.e. their expected ZT value is below the highest value in the dataset.  Therefore, they are biased towards materials with high model uncertainty as well.